# 1. Build Master Database

This notebook:
1. Reads raw CSV/Parquet from `data/raw/`
2. Cleans and filters
3. Engineers core feature flags
4. Writes out `data/processed/master.parquet`
5. Quick sanity checks


In [11]:
## 0. Import Libraries

import pandas as pd
import numpy as np
from pathlib import Path
# from ydata_profiling import ProfileReport
from datetime import datetime

pd.set_option('display.max_columns', 100)  
pd.set_option('display.width', None)       
pd.set_option('display.max_rows', 1000)

## 0 Load raw data and keep only August-2024 → today

In [12]:
RAW_DIR   = Path("../data/raw")
PROC_DIR  = Path("../data/processed")
RAW_CSV   = RAW_DIR / "Clays_data.csv"
RAW_PQ    = RAW_DIR / "full_raw.parquet"

# 0a.  Re-build parquet from the latest CSV (overwrites)
df = pd.read_csv(RAW_CSV)
df.to_parquet(RAW_PQ, index=False)
print(f"🔄  Rebuilt {RAW_PQ.name} from {RAW_CSV.name} ({len(df):,} rows)")

# 0b.  Work from the parquet for speed
df = pd.read_parquet(RAW_PQ)



# 0c.  keep Aug-24 and newer
df = df[~((df["Year"] == 2024) & (df["Month"] < 8))].reset_index(drop=True)

print(f"After date filter → {len(df):,} rows × {df.shape[1]} columns")
display(df.iloc[:3].T)      # transpose = easiest schema sanity-check


C:\Users\marti\AppData\Local\Temp\ipykernel_34024\219403485.py:7: DtypeWarning: Columns (14,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_CSV)


🔄  Rebuilt full_raw.parquet from Clays_data.csv (439,775 rows)
After date filter → 386,273 rows × 51 columns


,0,1,2
Context ID,20240801000316DQ6C,2024080100046GWKPO,202408010004Y09YOM
Booking ID,None,202408010004TRVEE4,202408010004YGI9KR
Session ID,81.98.105.115,81.98.105.115,81.98.105.115
Search At,2024-08-01T00:03:52.345,2024-08-01T00:04:39.622,2024-08-01T00:04:04.468
Search Date,2024-08-02,2024-08-02,2024-08-02
Search Time,64800000000000,64800000000000,64800000000000
Search Time Iso,18:00:00,18:00:00,18:00:00
Search Days Ahead,1,1,1
Search Charge,12.0,12.0,12.0
Search Charge Type,person,person,person


## 1 Drop the “red” columns

In [13]:
red_columns = [
    "Search Time",
    "Search Charge Type",
    "Venue ID",
    "Reservation Time",
    "Reservation Datetime",
    "Reservation Charge Type",
    "Game Area",
    "Was Package Required",
    "Marketing Opt In",
    "Reservation Notes",
    "Reservation ID",
    "Reservation Tags",    
]

df = df.drop(columns=[c for c in red_columns if c in df.columns])

print("Columns after drop →", len(df.columns))
display(df.iloc[:3].T) 

Columns after drop → 39


,0,1,2
Context ID,20240801000316DQ6C,2024080100046GWKPO,202408010004Y09YOM
Booking ID,None,202408010004TRVEE4,202408010004YGI9KR
Session ID,81.98.105.115,81.98.105.115,81.98.105.115
Search At,2024-08-01T00:03:52.345,2024-08-01T00:04:39.622,2024-08-01T00:04:04.468
Search Date,2024-08-02,2024-08-02,2024-08-02
Search Time Iso,18:00:00,18:00:00,18:00:00
Search Days Ahead,1,1,1
Search Charge,12.0,12.0,12.0
Venue Name,"Clays, The City",Clays Canary Wharf,Clays Canary Wharf
Party Size,2,2,2


## 2 Raw Cleaning & Filtering 



In [14]:
# ————————————————————————————————————————————————
# 0.  Up-front filters
# ————————————————————————————————————————————————

# 1 Replace dollars with pounds
df.columns = [col.replace('($)', '(£)') for col in df.columns]

# 2 Parse Search At safely (micro-seconds are fine!)
df["Search At"] = pd.to_datetime(df["Search At"], errors="coerce", utc=True)

# 3 Reservation Days Ahead → fill <NA> with 0
df["Reservation Days Ahead"] = (
    df["Reservation Days Ahead"]
      .fillna(0)           # or .replace("<NA>", 0) if dtype is object
      .astype("Int64")     # nullable integer (not float)
)

# 4 Convert flag columns from True/False → 0/1
flag_cols = [
    'Was Search Available',
    "Time Extension Available",
    "Time Extension Selected",
    "Personal Info Completed",
    "Promo Code Applied",
    "Billing Info Completed",
]

for col in flag_cols:
    # Safety: if the column is missing just skip it
    if col in df.columns:
        df[col] = df[col].fillna(False).astype(bool).astype("int8")


# ————————————————————————————————————————————————
# 1.  BUSINESS RULE FILTERS
# ————————————————————————————————————————————————
# 1a. max party size = 20
df = df[df["Party Size"] <= 20]

# 1b. remove negative money & lead-time values
money_cols = [
    "Search Charge",
    "Reservation Charge",
    "Reservation Cost (£)",
    "Packages Cost (£)",
    "Add Ons Cost (£)",
    "Promo Code Discount (£)",
    "Total Cost (£)",
    "Deposit Amount",
]
day_cols = ["Search Days Ahead", "Reservation Days Ahead"]

neg_check_cols = [c for c in money_cols + day_cols if c in df.columns]
df = df[~df[neg_check_cols].lt(0).any(axis=1)]

# 1c. keep only Search Days Ahead ≤ 180
df = df[df["Search Days Ahead"] <= 180]

# ————————————————————————————————————————————————
# 2.  DATETIME & COLUMN HOUSEKEEPING
# ————————————————————————————————————————————————
# 2a. split Search At into date & hour; drop the original
df["Search Date At"] = df["Search At"].dt.date
df["Search Hour At"] = df["Search At"].dt.strftime("%H:%M:%S")
df = df.drop(columns=["Search At"])

# 2b. rename the “for-date” columns and trim the hour
df = df.rename(
    columns={
        "Search Date": "Search Date For",
        "Search Time Iso": "Search Hour For",
        "Year": "Year At",
        "Month": "Month At",
    }
)

df["Search Hour For"] = df["Search Hour For"].astype(str).str[:2]  # '18' instead of '18:00:00'

# 2c. Was Booked = 1 if there is a real Booking ID
df["Started a Booking Draft"] = (
    df["Booking ID"].notna() & df["Booking ID"].ne("None")
).astype("int8")



# 2d. create new Was Booked (only successful reservations)
df["Was Booked"] = (df["Booking Status"] == "reservation_success").astype("int8")

# 2e. create duration_minutes
# Define conditions
conditions = [
    df['Party Size'].between(1, 2),
    df['Party Size'].between(3, 5),
    df['Party Size'].between(6, 20)
]

# Define corresponding values
values = [45, 60, 90]
df['duration_minutes'] = np.select(conditions, values, default=np.nan)  # or 0 if you'd rather mark unhandled sizes


# ————————————————————————————————————————————————
# 3. Creating "Day of the Week" column
# ————————————————————————————————————————————————

# Step 1: Define anchor
anchor_date = datetime.strptime("2024-08-01", "%Y-%m-%d").date()
anchor_weekday = 3  # 0=Monday, ..., 3=Thursday

# Step 2: Convert string to date manually
def parse_date_safely(date_str):
    """Parse date handling potential format variations"""
    try:
        # First try the actual format from your data
        return datetime.strptime(date_str, "%d/%m/%y").date()
    except ValueError:
        try:
            # Fallback to the original format if needed
            return datetime.strptime(date_str, "%Y-%m-%d").date()
        except ValueError:
            # If both fail, return None or handle as needed
            return None

df["Search Date Parsed"] = df["Search Date For"].apply(parse_date_safely)

# Check for any parsing failures
if df["Search Date Parsed"].isna().any():
    print(f"Warning: {df['Search Date Parsed'].isna().sum()} dates could not be parsed")
    # Optionally, you can inspect these problematic dates:
    # print(df[df["Search Date Parsed"].isna()]["Search Date For"].head())

# Step 3: Calculate offset in days from anchor
df["Days Since Anchor"] = df["Search Date Parsed"].apply(lambda x: (x - anchor_date).days)

# Step 4: Get Day of the Week (mod 7, add to anchor)
weekday_names = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df["Day of the Week"] = df["Days Since Anchor"].apply(lambda x: weekday_names[(anchor_weekday + x) % 7])

# Step 5: Create Is Weekend
df["Is Weekend"] = df["Day of the Week"].apply(lambda x: 1 if x in ["Saturday", "Sunday"] else 0)

# Step 6: Create remaining dates at/ for
df['Day At'] = pd.to_datetime(df['Search Date At']).dt.month
df['Year For'] = pd.to_datetime(df['Search Date For']).dt.year
df['Month For'] = pd.to_datetime(df['Search Date For']).dt.month
df['Day For'] = pd.to_datetime(df['Search Date For']).dt.day

# Optional cleanup
df.drop(columns=["Search Date Parsed", "Days Since Anchor"], inplace=True)



# ————————————————————————————————————————————————
# 4.  Full Ordered Column List
# ————————————————————————————————————————————————

ordered_columns = [
    # 1. Session & Context Info
    "Context ID", "Session ID", "Customer ID",

    # 2. Search Behavior (At) 
    "Search Date At", "Year At", "Month At", "Day At", "Search Hour At",
    
    # Search Behavior (For) & Time Dimensions
    "Search Date For", "Year For", "Month For", "Day For", "Search Hour For",
    "Day of the Week", "Is Weekend", "Search Days Ahead", "Was Search Available", 
    "Venue Name", "Party Size","Source Path", "Occasion ID", "Search Charge",

    # 3. Reservation Details
    "Reservation Date", "Reservation Time Iso", "Reservation Days Ahead",
    "Reservation Charge", "Booking ID", "Booking Status", "Reservation Reference Code",

    # 4. Packages and Add-Ons
    "Packages Available", "Packages", "Add Ons Available", "Add Ons",
    "Time Extension Available", "Time Extension Selected",

    # 5. Customer Checkout Info
    "Personal Info Completed", "Personal Info Completed At",
    "Billing Info Completed", "Promo Code", "Promo Code Applied",

    # 6. Financials
    "Reservation Cost (£)", "Packages Cost (£)", "Add Ons Cost (£)",
    "Promo Code Discount (£)", "Total Cost (£)", "Deposit Amount",

    # 7. Booking Funnel Events
    "Started a Booking Draft", "Was Booked",

    
]

df = df[ordered_columns]


# ————————————————————————————————————————————————
# 5.  Save cleaned data
# ————————————————————————————————————————————————
PROC_DIR.mkdir(parents=True, exist_ok=True)

CSV_PATH = PROC_DIR / "full_clean.csv"
PQ_PATH  = PROC_DIR / "full_clean.parquet"

df.to_csv(CSV_PATH, index=False)
df.to_parquet(PQ_PATH, index=False)

print(f"✅  Saved cleaned data → {CSV_PATH.name} & {PQ_PATH.name}")
print(f"   Final shape: {len(df):,} rows × {df.shape[1]} columns")


✅  Saved cleaned data → full_clean.csv & full_clean.parquet
   Final shape: 376,684 rows × 48 columns


In [15]:
# # ————————————————————————————————————————————————
# # QUICK DATA-QUALITY SNAPSHOT
# # ————————————————————————————————————————————————

# # % of missing & negative values — snapshot
# na_pct  = df.isna().mean().mul(100).round(2).sort_values(ascending=False)
# neg_pct = (df.select_dtypes("number").lt(0).mean()
#              .mul(100).round(2).sort_values(ascending=False))

# display(na_pct.to_frame("pct_missing").head(20))
# display(neg_pct.to_frame("pct_negative").head(20))


In [16]:
df.columns

Index(['Context ID', 'Session ID', 'Customer ID', 'Search Date At', 'Year At',
       'Month At', 'Day At', 'Search Hour At', 'Search Date For', 'Year For',
       'Month For', 'Day For', 'Search Hour For', 'Day of the Week',
       'Is Weekend', 'Search Days Ahead', 'Was Search Available', 'Venue Name',
       'Party Size', 'Source Path', 'Occasion ID', 'Search Charge',
       'Reservation Date', 'Reservation Time Iso', 'Reservation Days Ahead',
       'Reservation Charge', 'Booking ID', 'Booking Status',
       'Reservation Reference Code', 'Packages Available', 'Packages',
       'Add Ons Available', 'Add Ons', 'Time Extension Available',
       'Time Extension Selected', 'Personal Info Completed',
       'Personal Info Completed At', 'Billing Info Completed', 'Promo Code',
       'Promo Code Applied', 'Reservation Cost (£)', 'Packages Cost (£)',
       'Add Ons Cost (£)', 'Promo Code Discount (£)', 'Total Cost (£)',
       'Deposit Amount', 'Started a Booking Draft', 'Was Booked'],

In [17]:
display(df.head(3).T) 

,0,1,2
Context ID,20240801000316DQ6C,2024080100046GWKPO,202408010004Y09YOM
Session ID,81.98.105.115,81.98.105.115,81.98.105.115
Customer ID,None,202408010006KZKP9I,None
Search Date At,2024-08-01,2024-08-01,2024-08-01
Year At,2024,2024,2024
Month At,8,8,8
Day At,8.0,8.0,8.0
Search Hour At,00:03:52,00:04:39,00:04:04
Search Date For,2024-08-02,2024-08-02,2024-08-02
Year For,2024,2024,2024
